# Reading in Polars and Data

In [4]:
import polars as pl
pl.Config.with_columns_kwargs = True

In [6]:
batting = pl.read_csv('Data/lahman baseball data/Batting.csv')
pitching = pl.read_csv('Data/lahman baseball data/Pitching.csv')
fielding = pl.read_csv('Data/lahman baseball data/Fielding.csv')
awards = pl.read_csv('Data/lahman baseball data/AwardsPlayers.csv')
salaries = pl.read_csv('Data/lahman baseball data/Salaries.csv')
salaries

yearID,teamID,lgID,playerID,salary
i64,str,str,str,i64
1985,"""ATL""","""NL""","""barkele01""",870000
1985,"""ATL""","""NL""","""bedrost01""",550000
1985,"""ATL""","""NL""","""benedbr01""",545000
1985,"""ATL""","""NL""","""campri01""",633333
1985,"""ATL""","""NL""","""ceronri01""",625000
…,…,…,…,…
2016,"""WAS""","""NL""","""strasst01""",10400000
2016,"""WAS""","""NL""","""taylomi02""",524000
2016,"""WAS""","""NL""","""treinbl01""",524900


# Gold Glove Data Set

In [9]:
'case is player, year, pos, league'

'case is player, year, pos, league'

In [11]:
fielding_awards = (fielding
.filter((pl.col('yearID') >= 2013))
.join(
     awards.rename({"notes": "POS"}) 
           .select(['playerID', 'yearID', 'awardID', 'lgID', 'POS'])  
           .filter((pl.col('awardID') == 'Gold Glove') & (pl.col('yearID') >= 2013)),
      on=['playerID', 'yearID', 'lgID'],
      how='left'
     )
.group_by(['playerID','yearID', 'lgID', 'POS', 'awardID'])
.agg(
     pl.col('InnOuts').sum().alias('InnOuts'),
     pl.col('PO').sum().alias('PO'),
     pl.col('A').sum().alias('A'),
     pl.col('E').sum().alias('E'),
     pl.col('DP').sum().alias('DP'),
     pl.col('PB').cast(int).sum().alias('PB'),
     pl.col('WP').cast(int).sum().alias('WP'),
     pl.col('SB').cast(int).sum().alias('SB'),
     pl.col('CS').cast(int).sum().alias('CS'),
     pl.col('ZR').cast(int).sum().alias('ZR') 
    )    
.with_columns(
        pl.when(pl.col('awardID').is_null())
          .then(pl.lit('No'))
          .otherwise(pl.lit('Yes'))
          .alias('Gold Glove?'),
         pl.when(pl.col('yearID') == 2023)
          .then(pl.lit('Validation'))
          .otherwise(pl.lit('Training'))
          .alias('Training-Validation')
              )
.drop(pl.col('awardID'))
                  )
# fielding_awards = fielding_awards.filter(pl.col('Gold Glove?') == 'Yes', pl.col('POS') == 'C', pl.col('yearID') == 2017)
fielding_awards.write_csv('data/fielding_awards.csv')

fielding_awards

playerID,yearID,lgID,POS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR,Gold Glove?,Training-Validation
str,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str
"""cokeph01""",2013,"""AL""","""P""",115,0,6,1,0,0,0,0,0,0,"""No""","""Training"""
"""rayco01""",2021,"""NL""","""OF""",15,2,0,0,0,0,0,0,0,0,"""No""","""Training"""
"""sanchad01""",2019,"""NL""","""SS""",15,0,1,0,0,0,0,0,0,0,"""No""","""Training"""
"""taillja01""",2016,"""NL""","""P""",312,5,14,2,1,0,0,0,0,0,"""No""","""Training"""
"""bradfco01""",2023,"""AL""","""P""",168,1,3,0,0,0,0,0,0,0,"""No""","""Validation"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""raleylu01""",2023,"""AL""","""P""",8,0,0,0,0,0,0,0,0,0,"""No""","""Validation"""
"""castrda01""",2015,"""NL""","""3B""",57,3,3,1,3,0,0,0,0,0,"""No""","""Training"""
"""thomadi01""",2022,"""AL""","""OF""",102,9,0,0,0,0,0,0,0,0,"""No""","""Training"""


# Salary Dataset

In [14]:
'case is player year' 'salary, batting, fielding, pitching'

'case is player yearsalary, batting, fielding, pitching'

In [16]:
salaries_final = (salaries
.filter(pl.col('yearID') >= 2006)
.join(fielding
        .filter(pl.col('yearID') >= 2006),
        on=['playerID', 'yearID', 'lgID', 'teamID'],
        how='left'
     )
.join(batting
        .rename({"CS": "CS_batting", "SB": "SB_Batting"})
        .filter(pl.col('yearID') >= 2006),
        on=['playerID', 'yearID', 'teamID', 'stint', 'lgID', 'G'],
        how='left',
     )
.join(pitching
        .rename({
            "GIDP": "GIDP_pitching", "H": "H_pitching", "HR": "HR_pitching", "BB": "BB_pitching",
            "SO": "SO_pitching", "IBB": "IBB_pitching", "WP": "WP_pitching", "HBP": "HBP_pitching",
            "SH": "SH_pitching", "SF": "SF_pitching", "R": "R_pitching", "GIDP": "GIDP_pitching"
                })
        .filter(pl.col('yearID') >= 2006),
        on=['playerID', 'yearID', 'teamID', 'stint', 'lgID', 'G', 'GS'],
        how='left'
     )
.group_by(['playerID','yearID'])
.agg(
     pl.col('salary').sum().alias('salary'),
     pl.col('InnOuts').sum().alias('InnOuts'),
     pl.col('PO').sum().alias('PO'),
     pl.col('A').sum().alias('A'),
     pl.col('E').sum().alias('E'),
     pl.col('DP').sum().alias('DP'),
     pl.col('PB').cast(int).sum().alias('PB'),
     pl.col('WP').cast(int).sum().alias('WP'),
     pl.col('SB').cast(int).sum().alias('SB'),
     pl.col('CS').cast(int).sum().alias('CS'),
     pl.col('ZR').cast(int).sum().alias('ZR'),
     pl.col('G').sum().alias('G'),
     pl.col('AB').sum().alias('AB'),
     pl.col('R').sum().alias('R'),
     pl.col('H').sum().alias('H'),
     pl.col('2B').sum().alias('2B'),
     pl.col('3B').sum().alias('3B'),
     pl.col('HR').sum().alias('HR'),
     pl.col('SB_Batting').sum().alias('SB_batting'),
     pl.col('CS_batting').sum().alias('CS_batting'),
     pl.col('BB').sum().alias('BB_batting'),
     pl.col('SO').sum().alias('SO_batting'),
     pl.col('IBB').sum().alias('IBB_batting'),
     pl.col('HBP').sum().alias('HBP_batting'),
     pl.col('SH').sum().alias('Sacrifice Hits'),
     pl.col('SF').sum().alias('Sacrifice Flies'),
     pl.col('GIDP').sum().alias('GIDP'),
     pl.col('GS').sum().alias('GS'),
     pl.col('CG').sum().alias('CG'),
     pl.col('SHO').sum().alias('SHO'),
     pl.col('SV').sum().alias('SV'),
     pl.col('IPouts').sum().alias('IPOuts'),
     pl.col('H_pitching').sum().alias('H_pitching'),
     pl.col('ER').sum().alias('ER_pitching'),
     pl.col('HR_pitching').sum().alias('HR_pitching'),
     pl.col('BB_pitching').sum().alias('BB_pitching'),
     pl.col('SO_pitching').sum().alias('SO_pitching'),
     pl.col('BAOpp').sum().alias('BAOpp'),
     pl.col('ERA').sum().alias('ERA_pitching'),
     pl.col('IBB_pitching').sum().alias('IBB_pitching'),
     pl.col('WP_pitching').sum().alias('WP_pitching'),
     pl.col('HBP_pitching').sum().alias('HBP_pitching'),
     pl.col('BK').sum().alias('Balk'),
     pl.col('BFP').sum().alias('Batters Faced by Pitcher'),
     pl.col('GF').sum().alias('Games Finished'),
     pl.col('SH').sum().alias('SH_pitching'),
     pl.col('SF').sum().alias('SF_pitching'),
     pl.col('GIDP_pitching').sum().alias('GIDP_pitching')
    )   
.with_columns(
         pl.when(pl.col('yearID') == 2016)
          .then(pl.lit('Validation'))
          .otherwise(pl.lit('Training'))
          .alias('Training-Validation')
               )
)
salaries_final.write_csv('data/salaries_final.csv')
salaries_final

playerID,yearID,salary,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR,G,AB,R,H,2B,3B,HR,SB_batting,CS_batting,BB_batting,SO_batting,IBB_batting,HBP_batting,Sacrifice Hits,Sacrifice Flies,GIDP,GS,CG,SHO,SV,IPOuts,H_pitching,ER_pitching,HR_pitching,BB_pitching,SO_pitching,BAOpp,ERA_pitching,IBB_pitching,WP_pitching,HBP_pitching,Balk,Batters Faced by Pitcher,Games Finished,SH_pitching,SF_pitching,GIDP_pitching,Training-Validation
str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,f64,f64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str
"""ruschgl01""",2009,750000,56,1,2,0,0,0,0,0,0,0,11,6,0,2,1,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,56,35,14,3,3,13,0.398,6.75,1,0,0,0,92,1,0,0,3,"""Training"""
"""sanchbr01""",2011,425000,185,3,3,0,0,0,0,0,0,0,39,5,0,1,0,0,0,0,0,0,2,0,0,0,0,1,2,0,0,0,185,52,27,7,36,53,0.227,3.94,6,4,2,1,270,8,0,0,3,"""Training"""
"""ariasjo01""",2015,5800000,288,22,34,4,6,0,0,0,0,0,20,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,9,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,"""Training"""
"""garlajo01""",2009,6250000,503,18,28,4,3,0,0,0,0,0,27,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,27,1,0,0,503,188,80,19,52,83,0.286,4.29,5,5,6,0,728,0,0,0,22,"""Training"""
"""ariasjo01""",2014,4600000,1251,132,85,3,26,0,0,0,0,0,86,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,39,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,"""Training"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""dejesiv02""",2011,414000,188,12,21,1,2,0,0,0,0,0,11,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,7,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,"""Training"""
"""heltoto01""",2010,17775000,2843,932,77,8,116,0,0,0,0,0,115,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,105,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,"""Training"""
"""greenkh01""",2006,405000,2993,138,309,9,60,0,0,0,0,0,113,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,111,0,0,0,0,0,0,0,0,0,0.0,0.0,0,0,0,0,0,0,0,0,0,"""Training"""


# Classification Problem

## Setting Up

In [18]:
import sklearn
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [20]:
from sklearn.metrics import confusion_matrix

def get_metrics(y_true, y_pred, labels=None):
    """ Compute various measures from a confusion matrix

    Args:
        y_true: a column of the true labels.
        y_pred: a column of the predicted labels.
    Returns:
        A dictionary of metrics.
    """
    conf_mat = confusion_matrix(y_true, y_pred, labels=labels)
    tn, fp, fn, tp = conf_mat.ravel()

    return {'accuracy'          :  (tp + tn)/(tp + tn + fp + fn),
            'balanced_accuracy' :  0.5*tn / (tn+fp) + 0.5*tp / (tp+fn),
            'specificity'       :  tn / (tn+fp),
            'sensitivity'       :  tp / (tp+fn),
             'ppv'              :  tp / (tp + fp),
             'npv'              :  tn / (tn + fn)}


In [23]:
fielding_awards

playerID,yearID,lgID,POS,InnOuts,PO,A,E,DP,PB,WP,SB,CS,ZR,Gold Glove?,Training-Validation
str,i64,str,str,i64,i64,i64,i64,i64,i64,i64,i64,i64,i64,str,str
"""topaju01""",2022,"""NL""","""P""",22,1,4,0,0,0,0,0,0,0,"""No""","""Training"""
"""whitemi03""",2022,"""NL""","""P""",168,5,4,0,0,0,0,0,0,0,"""No""","""Training"""
"""andergr01""",2023,"""AL""","""P""",107,3,4,0,1,0,0,0,0,0,"""No""","""Validation"""
"""madrini01""",2022,"""NL""","""2B""",1462,106,144,1,46,0,0,0,0,0,"""No""","""Training"""
"""doteloc01""",2013,"""AL""","""P""",14,0,1,0,0,0,0,0,0,0,"""No""","""Training"""
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""stallja01""",2023,"""NL""","""P""",24,0,0,0,0,0,0,0,0,0,"""No""","""Validation"""
"""anderti01""",2023,"""AL""","""2B""",54,3,2,1,0,0,0,0,0,0,"""No""","""Validation"""
"""castrja01""",2020,"""AL""","""C""",383,131,7,0,0,1,0,7,4,0,"""No""","""Training"""


In [25]:
fielding_awards.columns

['playerID',
 'yearID',
 'lgID',
 'POS',
 'InnOuts',
 'PO',
 'A',
 'E',
 'DP',
 'PB',
 'WP',
 'SB',
 'CS',
 'ZR',
 'Gold Glove?',
 'Training-Validation']

## Preparing Data

In [28]:
fielding_training = fielding_awards.filter(pl.col('Training-Validation') == 'Training').drop(['Gold Glove?', 'playerID', 'yearID', 'lgID', 'POS', 'Training-Validation'])
gold_glove_train = fielding_awards.filter(pl.col('Training-Validation') == 'Training').drop(['playerID', 'yearID', 'lgID', 'POS', 'Training-Validation']).select('Gold Glove?')

fielding_validation = fielding_awards.filter(pl.col('Training-Validation') == 'Validation').drop(['Gold Glove?', 'playerID', 'yearID', 'lgID', 'POS', 'Training-Validation'])
gold_glove_validation = fielding_awards.filter(pl.col('Training-Validation') == 'Validation').drop(['playerID', 'yearID', 'lgID', 'POS', 'Training-Validation']).select('Gold Glove?')

fielding_stats_train = fielding_training.to_pandas()
gold_glove_train = gold_glove_train.to_pandas()

fielding_stats_validation = fielding_validation.to_pandas()
gold_glove_validation = gold_glove_validation.to_pandas()

## Grid Search

### Tree Grid Search

In [24]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
import numpy as np

In [32]:
param_grid = {'max_depth': [None, 1,2,3,4,5],
              'min_samples_split' : [1, 5, 10],
               'min_samples_leaf' : [1, 5, 10],
              'class_weight':[None, "balanced"]
              }


In [26]:
from sklearn.model_selection import StratifiedKFold

In [ ]:
state = 458

cv_obj = StratifiedKFold(n_splits=10, shuffle=True, random_state=state)

grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid, verbose=3, cv=cv_obj)

grid_search.fit(fielding_training, gold_glove_train)

In [38]:
grid_search.best_score_

0.9881267356293654

In [40]:
grid_search.best_params_

{'class_weight': None,
 'max_depth': 1,
 'min_samples_leaf': 1,
 'min_samples_split': 5}

In [42]:
best_model_tree = grid_search.best_estimator_
best_model_tree

DecisionTreeClassifier(max_depth=1, min_samples_split=5)

### Random Forest Grid Search

In [45]:
rf_param_grid = {'n_estimators': 10 ** np.arange(1, 3),
              'max_depth': [None, 1,2,3,4,5],
              'min_samples_split' : [1, 5, 10],
               'min_samples_leaf' : [1, 5, 10],
              'class_weight':[None, "balanced"]
              }

rf_param_grid

{'n_estimators': array([ 10, 100], dtype=int32),
 'max_depth': [None, 1, 2, 3, 4, 5],
 'min_samples_split': [1, 5, 10],
 'min_samples_leaf': [1, 5, 10],
 'class_weight': [None, 'balanced']}

In [10]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
rf_grid_search = GridSearchCV(RandomForestClassifier(), rf_param_grid, verbose=3, cv=cv_obj)

rf_grid_search.fit(fielding_training, gold_glove_train)

In [ ]:
rf_grid_search.best_score_

In [ ]:
rf_grid_search.best_params_

In [ ]:
best_model_rf = rf_grid_search.best_estimator_
best_model_rf

### Comparing Scores

In [ ]:
rf_grid_search.best_score_ > grid_search.best_score_

### Measuring Performance

In [ ]:
gold_glove_validation_pred_rf = rf_grid_search.predict(fielding_validation)

In [ ]:
get_metrics(gold_glove_validation, gold_glove_validation_pred_rf)

## Fitting the Best Model

In [ ]:
forest = RandomForestClassifier(min_samples_leaf=5, min_samples_split=10, n_estimators=10)
forest.fit(fielding_training, gold_glove_train)

## ROC Curve

In [ ]:
from sklearn.metrics import RocCurveDisplay

plt.rcParams["figure.figsize"] = (8,6)

RocCurveDisplay.from_estimator(forest, fielding_training, gold_glove_train)

plt.show()

## AUC

In [11]:
def get_class_prob_df(clf, X):
    """ Return a dataframe of class probabilities.

    Arg:
        clf:  A fitted model object
        X:    The matrix of (test) features.
    Returns: A dataframe of class probabilities
    """
    return pd.DataFrame(clf.predict_proba(X), columns=clf.classes_)

In [ ]:
y_test_probs = get_class_prob_df(forest, fielding_validation)

y_test_probs

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(gold_glove_validation, y_test_probs['Yes'])

## Balanced Success

In [ ]:
forest = RandomForestClassifier(min_samples_leaf=5, min_samples_split=10, n_estimators=10)
forest.fit(fielding_training, gold_glove_train)

y_test_pred = forest.predict(fielding_validation)
y_test_prob = get_class_prob_df(forest, fielding_validation)

In [ ]:
forest2 = RandomForestClassifier()
forest2.fit(fielding_training, gold_glove_train)

y_test_pred2 = forest.predict(fielding_validation)
y_test_prob2 = get_class_prob_df(forest2, fielding_validation)

In [ ]:
get_metrics(gold_glove_validation, y_test_pred)

In [ ]:
get_metrics(gold_glove_validation, y_test_pred2)

In [ ]:
ax = plt.gca()

RocCurveDisplay.from_estimator(forest, fielding_validation, gold_glove_validation, ax = ax, name = "Random Forest")
RocCurveDisplay.from_estimator(forest2, fielding_validation, gold_glove_validation, ax = ax, name = "Default Random Forest")

plt.show()

In [ ]:
roc_auc_score(gold_glove_validation, y_test_probs['Yes'])

In [ ]:
roc_auc_score(gold_glove_validation, y_test_prob2['Yes'])

# Regression Problem

## Preparing Data

In [28]:
salaries.columns

['yearID', 'teamID', 'lgID', 'playerID', 'salary']

In [30]:
salaries_training = salaries_final.filter(pl.col('Training-Validation') == 'Training').drop('playerID','yearID','salary', 'Training-Validation')
salary_train = salaries_final.filter(pl.col('Training-Validation') == 'Training').drop('playerID','yearID','Training-Validation').select('salary')

salaries_validation = salaries_final.filter(pl.col('Training-Validation') == 'Validation').drop('playerID','yearID','salary', 'Training-Validation')
salary_validation = salaries_final.filter(pl.col('Training-Validation') == 'Validation').drop('playerID','yearID','Training-Validation').select('salary')

salaries_training = salaries_training.to_pandas()
salary_train = salary_train.to_pandas()

salaries_validation = salaries_validation.to_pandas()
salary_validation = salary_validation.to_pandas()


In [32]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

## Tree Grid Search

In [35]:
tree_param_grid = {
    'max_depth': [None, 3, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10]
}

from sklearn.model_selection import StratifiedKFold

state = 458

cv_obj = StratifiedKFold(n_splits=10, shuffle=True, random_state=state)

grid_search_tree_regressor = GridSearchCV(DecisionTreeRegressor(), tree_param_grid, verbose=3, cv=cv_obj)

grid_search_tree_regressor.fit(salaries_training, salary_train)

Fitting 10 folds for each of 36 candidates, totalling 360 fits
[CV 1/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.420 total time=   0.0s


C:\Users\ben8h\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(


[CV 2/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.934 total time=   0.0s
[CV 3/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.493 total time=   0.0s
[CV 4/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.484 total time=   0.0s
[CV 5/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.514 total time=   0.0s
[CV 6/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.574 total time=   0.0s
[CV 7/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.709 total time=   0.0s
[CV 8/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.602 total time=   0.0s
[CV 9/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.610 total time=   0.0s
[CV 10/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2;, score=-0.580 total time=   0.0s
[CV 1/10] END max_depth=None, min_samples_leaf=1, min_samples_s

GridSearchCV(cv=StratifiedKFold(n_splits=10, random_state=458, shuffle=True),
             estimator=DecisionTreeRegressor(),
             param_grid={'max_depth': [None, 3, 5, 10],
                         'min_samples_leaf': [1, 5, 10],
                         'min_samples_split': [2, 5, 10]},
             verbose=3)

In [37]:
grid_search_tree_regressor.best_score_

0.1226222319983796

In [39]:
grid_search_tree_regressor.best_params_

{'max_depth': 5, 'min_samples_leaf': 10, 'min_samples_split': 2}

In [41]:
best_model_tree_regressor = grid_search_tree_regressor.best_estimator_
best_model_tree_regressor

DecisionTreeRegressor(max_depth=5, min_samples_leaf=10)

## Forest Grid Search

In [ ]:
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 5, 10],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 5, 10]
}

from sklearn.model_selection import StratifiedKFold

state = 458

cv_obj = StratifiedKFold(n_splits=10, shuffle=True, random_state=state)

grid_search_forest_r = GridSearchCV(RandomForestRegressor(), rf_param_grid, verbose=3, cv=cv_obj)

grid_search_forest_r.fit(salaries_training, salary_train)

Fitting 10 folds for each of 81 candidates, totalling 810 fits


C:\Users\ben8h\anaconda3\Lib\site-packages\sklearn\model_selection\_split.py:737: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=10.
  warnings.warn(
C:\Users\ben8h\anaconda3\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV 1/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.180 total time=   2.9s


C:\Users\ben8h\anaconda3\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV 2/10] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=50;, score=0.068 total time=   2.8s


...

C:\Users\ben8h\anaconda3\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV 2/10] END max_depth=None, min_samples_leaf=10, min_samples_split=5, n_estimators=200;, score=0.171 total time=   6.3s


C:\Users\ben8h\anaconda3\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


[CV 3/10] END max_depth=None, min_samples_leaf=10, min_samples_split=5, n_estimators=200;, score=0.153 total time=   6.3s


C:\Users\ben8h\anaconda3\Lib\site-packages\sklearn\base.py:1474: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


In [ ]:
grid_search_forest_r.best_score_

In [ ]:
grid_search_forest_r.best_params_

In [ ]:
best_model_forest_r = grid_search_forest_r.best_estimator_
best_model_forest_r

## Comparing Scores

In [ ]:
grid_search_forest_r.best_score_ > grid_search_tree_regressor.best_score_

## Measuring Performance

In [ ]:
salaries_validation_pred_rf = grid_search_forest_r.predict(salaries_validation)

In [ ]:
get_metrics(salary_validation, salaries_validation_pred_rf)

## Fitting the Best Model

In [ ]:
forest = RandomForestClassifier(min_samples_leaf=5, min_samples_split=10, n_estimators=10)
forest.fit(salaries_training, salary_train)

## Root MSE & $R^2$

In [ ]:
from sklearn.metrics import explained_variance_score, mean_absolute_error, mean_squared_error

In [ ]:
salary_validation_pred = forest.predict(salaries_validation)

salary_validation_pred

In [ ]:
{'R^2': explained_variance_score(salary_validation, salary_validation_pred),
 'MSE': mean_squared_error(salary_validation, salary_validation_pred)
}